In [ ]:
import pandas as pd
from geopy.distance import great_circle


In [ ]:
def load_csv(file_path, encoding='latin1', sep=','):
    return pd.read_csv(file_path, encoding=encoding, sep=sep)

def load_excel(file_path, sheet_name=None):
    return pd.read_excel(file_path, sheet_name=sheet_name)

In [ ]:
def preprocess_geocode(geocode_df):
    geocode_df.dropna(subset=['latitude', 'longitude'], inplace=True)
    geocode_df['latitude'] = pd.to_numeric(geocode_df['latitude'])
    geocode_df['longitude'] = pd.to_numeric(geocode_df['longitude'])
    return geocode_df

In [ ]:
def find_closest_municipio(company_lat, company_lon, municipio_df):
    min_distance = float('inf')
    closest_municipio = None
    for _, municipio in municipio_df.iterrows():
        municipio_lat = municipio['latitud']
        municipio_lon = municipio['longitud']
        distance = great_circle((company_lat, company_lon), (municipio_lat, municipio_lon)).kilometers
        if distance < min_distance:
            min_distance = distance
            closest_municipio = municipio
    return closest_municipio, min_distance

def match_companies_to_municipio(companies_df, municipio_df):
    matched_df = companies_df.copy()
    matched_df['municipio'] = None
    matched_df['distance_to_municipio'] = float('inf')
    for index, company in companies_df.iterrows():
        closest_municipio, distance = find_closest_municipio(
            company['latitude'], company['longitude'], municipio_df
        )
        matched_df.at[index, 'municipio'] = closest_municipio['municipio']
        matched_df.at[index, 'distance_to_municipio'] = distance
    return matched_df

In [ ]:
def process_dates(df):
    df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y')
    df['Ano'] = df['date'].dt.year
    return df

def join_with_max_year(df1, df2, left_on, right_on, year_column):
    result_df = pd.merge(df1, df2, left_on=left_on, right_on=right_on, how='left')
    most_recent_df = df2.loc[df2.groupby(right_on[1])[year_column].idxmax()].drop(columns=year_column)
    fallback_df = pd.merge(df1[left_on[1:]], most_recent_df, left_on=left_on[1:], right_on=right_on[1:], how='left')
    final_df = result_df.combine_first(fallback_df)
    return final_df

def join_dfs(df1, df2, left_columns, right_columns):
    if len(left_columns) != len(right_columns):
        raise ValueError("The lengths of left_columns and right_columns must match.")
    result_df = pd.merge(df1, df2, left_on=left_columns, right_on=right_columns, how='left')
    return result_df

In [ ]:
def save_to_csv(df, file_path, encoding='latin1'):
    df.to_csv(file_path, index=False, encoding=encoding)

In [ ]:
# Load data
geocode = load_csv('/mnt/c/Users/clayt/Data Science/UCM/TFM/Datos/Processed/data_valencia_geocoded_final_survival.csv')
municipios = load_csv('/mnt/c/Users/clayt/Data Science/UCM/TFM/Datos/Raw/valencia_municipios.csv')
poblacion = load_excel('/mnt/c/Users/clayt/Data Science/UCM/TFM/Datos/Raw/poblacion_municipios_valencia.xlsx')
paro = load_excel('/mnt/c/Users/clayt/Data Science/UCM/TFM/Datos/Raw/paro_municipio_sector.xlsx')

# Preprocess geocode data
geocode = preprocess_geocode(geocode)

# Match companies to municipalities
geocode_municipio = match_companies_to_municipio(geocode, municipios)

# Process dates
geocode_municipio = process_dates(geocode_municipio)

# Join with population data
geocode_mun_pob = join_with_max_year(geocode_municipio, poblacion, left_on=['Ano', 'municipio'], right_on=['Ano', 'Municipio'], year_column='Ano')
geocode_mun_pob.drop(columns=['Municipio'], inplace=True)

# Join with unemployment data
geocode_mun_pob_paro = join_dfs(geocode_mun_pob, paro, left_columns=['Ano', 'municipio'], right_columns=['Ano', 'Municipio'])
geocode_mun_pob_paro.drop(columns=['Municipio'], inplace=True)

# Drop rows with NaN values
geocode_mun_pob_paro_final = geocode_mun_pob_paro.dropna()

# Save the final DataFrame
save_to_csv(geocode_mun_pob_paro_final, '/mnt/c/Users/clayt/Data Science/UCM/TFM/Datos/Processed/valencia_geocoded_pob_paro_survival.csv')